## konlpy Test

In [5]:
from konlpy.tag import Kkma
a =Kkma()
nouns=a.nouns("명사만을 추출합니다. 형태소 분석입니다.")
print(nouns)

['명사', '추출', '형태소', '분석']


## 네이버 기사 스크래핑 (Selinum, BeautifulSoup 이용)

In [1]:
import pandas as pd
import numpy as np


In [2]:
# -*- coding: UTF-8 -*- 
# [ IMPORT ] #
import urllib.request
import requests
from bs4 import BeautifulSoup as bs
import pymysql
from  selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time 

### 기사에 불필요한 특수 문자 혹은 문구 제거

In [3]:
import re
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]' , '', text)
    #cleaned_text = re.sub('[^0-9]',
                        #  '', cleaned_text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"\【\】]',
                          '', cleaned_text)
    cleaned_text = re.sub('[\▲\ⓒ\n]',
                          '', cleaned_text)
    cleaned_text = re.sub('오류를 우회하기 위한 함수 추가',
                          '', cleaned_text)

    cleaned_text.lstrip()
    cleaned_text.rstrip()
    return cleaned_text


In [215]:
data = pd.DataFrame() 
data['article'] = None
data['label'] = None

In [5]:
# 네이버-정치 기사 스크래핑 정치 -> label =0, 2020-04-14, 2020-04-17

driver = webdriver.Chrome("C://Users//kse02//Downloads//chromedriver_win32//chromedriver.exe")
driver.get("https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=100")
driver.implicitly_wait(5)

html = driver.page_source
soup = bs(html, 'html.parser')

links = []
for link in soup.findAll("a" , class_ = 'cluster_text_headline nclicks(cls_pol.clsart)'):
    links.append(link.attrs['href'])
links = np.array(links)


data_temp = pd.DataFrame()
data_temp = data_temp.append({'article':'','label':''}, ignore_index=True)
for i in range(0, len(links)):
    driver.get(links[i])
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    #article = clean_text(soup.find('div', class_ = '_article_body_contents').get_text())
    
    data_temp.iat[0,data.columns.get_loc('article')]= clean_text(soup.find('div', class_ = '_article_body_contents').get_text())
    data_temp.iat[0,data.columns.get_loc('label')] = 0
    
    data = data.append(data_temp, ignore_index = True)
    time.sleep(3)
    
    

In [6]:
# 네이버-경제 기사 스크래핑 경제 -> label =1, 2020-04-14, 2020-04-17

driver = webdriver.Chrome("C://Users//kse02//Downloads//chromedriver_win32//chromedriver.exe")
driver.get("https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=101")
driver.implicitly_wait(5)

html = driver.page_source
soup = bs(html, 'html.parser')

links = []
for link in soup.findAll("a" , class_ = 'cluster_text_headline nclicks(cls_eco.clsart)'):
    links.append(link.attrs['href'])
links = np.array(links)

data_temp = pd.DataFrame()
data_temp = data_temp.append({'article':'','label':''}, ignore_index=True)
for i in range(0, len(links)):
    driver.get(links[i])
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    #article = clean_text(soup.find('div', class_ = '_article_body_contents').get_text())
    
    data_temp.iat[0,data.columns.get_loc('article')]= clean_text(soup.find('div', class_ = '_article_body_contents').get_text())
    data_temp.iat[0,data.columns.get_loc('label')] = 1
    
    data = data.append(data_temp, ignore_index = True)
    time.sleep(3)
    
    

In [7]:
data

,article,label
0,“의료물품 비축제도 신설…기금 등 재원 총동원”서울신문“ 올해 서명되면 큰 ...,0
1,\t\t의료물품 비축제도 신설…인도적 지원 최대한 협조 기금 등 재원 모두 ...,0
2,아세안3 특별 화상 정상회의서 채택…문대통령 모두 및 의제 발언아이뉴스24 ...,0
3,문재인 대통령이 14일 오후 청와대 본관 집무실에서 코로나19 공동 대응...,0
4,차명진 후보 연합뉴스 법원의 결정에 따라 4·15 총선 하루 전...,0
...,...,...
91,영구임대·국민임대·행복주택… 단순화해 ‘통합공공임대’로 올해는 과천·남양주...,1
92,동영상 뉴스 \t\t 코로나19로 글로벌 경제가 위기에 빠지면서 각국 중앙은행...,1
93,금통위 친정부 인사 대거 포진한은 쪽 인사 과반 이상 평가도코로나19 경제위...,1
94,사진이미지투데이국제유가가 18년 만에 최저치를 기록했다 신종 코로나바이러스 ...,1


In [8]:
data.to_csv('data.csv',index=False)

In [4]:
data = pd.read_csv('data.csv')
data

,article,label
0,“의료물품 비축제도 신설…기금 등 재원 총동원”서울신문“ 올해 서명되면 큰 ...,0
1,\t\t의료물품 비축제도 신설…인도적 지원 최대한 협조 기금 등 재원 모두 ...,0
2,아세안3 특별 화상 정상회의서 채택…문대통령 모두 및 의제 발언아이뉴스24 ...,0
3,문재인 대통령이 14일 오후 청와대 본관 집무실에서 코로나19 공동 대응...,0
4,차명진 후보 연합뉴스 법원의 결정에 따라 4·15 총선 하루 전...,0
5,선관위 “총선 후보자 등록 무효처분 취소”서울신문법원 “차명진 제명 처리 절...,0
6,\t\t법원 제명결의 무효 가처분신청 인용…윤리위 안 거친 최고위 제명 무효...,0
7,\t\t머니투데이 박종진 기자 300서울뉴스1 김명섭 기자 황교안 미래통...,0
8,ㆍ2017년 이후 약 3년 만ㆍ‘북한판 접근거부 전략’인 듯ㆍ전투기 로켓 공...,0
9,동영상 뉴스 \t\t앵커북한이 오늘 아침 동해 상으로 단거리 순항미사일 추정 ...,0


## test 데이터 스크래핑

In [4]:
data1 = pd.DataFrame() 
data1['article'] = None
data1['label'] = None

In [6]:
# 네이버-정치 기사 스크래핑 정치 -> label =0, 2020-04-16

driver = webdriver.Chrome("C://Users//kse02//Downloads//chromedriver_win32//chromedriver.exe")
driver.get("https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=100")
driver.implicitly_wait(5)

html = driver.page_source
soup = bs(html, 'html.parser')

links = []
for link in soup.findAll("a" , class_ = 'cluster_text_headline nclicks(cls_pol.clsart)'):
    links.append(link.attrs['href'])
links = np.array(links)


data_temp = pd.DataFrame()
data_temp = data_temp.append({'article':'','label':''}, ignore_index=True)
for i in range(0, len(links)):
    driver.get(links[i])
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    #article = clean_text(soup.find('div', class_ = '_article_body_contents').get_text())
    
    data_temp.iat[0,data1.columns.get_loc('article')]= clean_text(soup.find('div', class_ = '_article_body_contents').get_text())
    data_temp.iat[0,data1.columns.get_loc('label')] = 0
    
    data1 = data1.append(data_temp, ignore_index = True)
    time.sleep(3)
    

In [7]:
# 네이버-경제 기사 스크래핑 경제 -> label =1, 2020-04-16

driver = webdriver.Chrome("C://Users//kse02//Downloads//chromedriver_win32//chromedriver.exe")
driver.get("https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=101")
driver.implicitly_wait(5)

html = driver.page_source
soup = bs(html, 'html.parser')

links = []
for link in soup.findAll("a" , class_ = 'cluster_text_headline nclicks(cls_eco.clsart)'):
    links.append(link.attrs['href'])
links = np.array(links)

data_temp = pd.DataFrame()
data_temp = data_temp.append({'article':'','label':''}, ignore_index=True)
for i in range(0, len(links)):
    driver.get(links[i])
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    #article = clean_text(soup.find('div', class_ = '_article_body_contents').get_text())
    
    data_temp.iat[0,data1.columns.get_loc('article')]= clean_text(soup.find('div', class_ = '_article_body_contents').get_text())
    data_temp.iat[0,data1.columns.get_loc('label')] = 1
    
    data1 = data1.append(data_temp, ignore_index = True)
    time.sleep(3)
    
    

In [8]:
data1

,article,label
0,눈물 닦는 심상정 사진연합뉴스정의당 심상정 대표는 오늘16일 고단한 정의당...,0
1,동영상 뉴스 국민의선택정의당 심상정 대표는 오늘16일 고단한 정의당의 길을 ...,0
2,정의당 선대위 해단식서 눈물 심상정 “20년 고된 길 다시 새롭게 시작하겠다...,0
3,제21대 총선에서 기대에 한참 못 미치는 성적표를 받아든 정의당 심상정 대표...,0
4,\t\t서울신문북한 간부들 태양절 맞아 금수산궁전 참배 북한 고위간부들이 ...,0
5,\t\t서울연합뉴스 박수윤 기자 북한 고위간부들이 김일성 주석의 108회 ...,0
6,북한 고위 간부들만 금수산태양궁전 참배김정은 공개 활동 없이 조화만 포착건강...,0
7,\t\t머니투데이 권다희 기자 300평양 노동신문뉴스1 북한 노동당과 정...,0
8,사진 출처 세월호 관련 막말 논란을 일으켰던 차명진 미래통합당 경기 부천병...,0
9,제21대 총선 경기 부천병에 출마한 미래통합당 차명진 후보가 지난 10일 오...,0


In [9]:
data1.to_csv('data1.csv',index=False)